In [ ]:
import json
from urllib import request
import pathlib
import zipfile
import os

import pymedphys

In [ ]:
task = 'Task06_Lung'

In [ ]:
data_root = pymedphys._config.get_config_dir().joinpath('data')

nnUNet_data_root = data_root.joinpath('nnUNet')
nnUNet_data_root.mkdir(exist_ok=True, parents=True)

nnUNet_raw_data_base = nnUNet_data_root.joinpath('nnUNet_raw_data_base')

raw_data_dir = nnUNet_raw_data_base.joinpath('nnUNet_raw_data')
raw_data_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
relative_raw_data_dir = raw_data_dir.relative_to(data_root)
relative_raw_data_dir

In [ ]:
here = pathlib.Path('.')

nnUNet_preprocessed = here.joinpath('nnUNet_preprocessed')
results_folder = here.joinpath('results')

In [ ]:
hash_path = nnUNet_data_root.joinpath('hashes.json')

if not hash_path.exists():
    with open(hash_path, "w") as f:
        f.write("{}")

In [ ]:
base_download_url = 'https://github.com/pymedphys/data/releases/download'

In [ ]:
def get_metadata_path(task):
    return relative_raw_data_dir.joinpath(task, "dataset.json")

In [ ]:
def get_metadata(task):
    download_url = f"{base_download_url}/{task}/dataset.json"
    metadata_path = pymedphys.data_path(
        get_metadata_path(task),
        url=download_url,
        hash_filepath=hash_path
    )
    
    with open(metadata_path) as f:
        metadata = json.load(f)
        
    return metadata

In [ ]:
def download_task_path(task, path):
    url = f"{base_download_url}/{task}/{path[2:].replace('/', '--os.sep--')}"
    full_path = relative_raw_data_dir.joinpath(task).joinpath(path)
    
    return pymedphys.data_path(
        full_path,
        url=url,
        hash_filepath=hash_path,
        delete_when_no_hash_found=False
    )


def get_filepaths_for_task(task):
    metadata = get_metadata(task)
        
    for paths in metadata['training']:
        image_path = download_task_path(task, paths['image'])
        label_path = download_task_path(task, paths['label'])
        
        yield image_path, label_path

In [ ]:
gen = get_filepaths_for_task(task)
list(gen);

In [ ]:
task_path = raw_data_dir.joinpath(task)
task_path

In [ ]:
metadata_path = data_root.joinpath(get_metadata_path(task))

In [ ]:
metadata = get_metadata(task)
metadata['test'] = []
# metadata

In [ ]:
with open(metadata_path, 'w') as f:
    json.dump(metadata, f)

In [ ]:
ts_path = task_path.joinpath('imagesTs')
ts_path.mkdir(exist_ok=True)

In [ ]:
os.environ["nnUNet_raw_data_base"] = str(nnUNet_raw_data_base)
os.environ["nnUNet_preprocessed"] = str(nnUNet_preprocessed)
os.environ["RESULTS_FOLDER"] = str(results_folder)

In [ ]:
!nnUNet_convert_decathlon_task -i {str(task_path)} -p 1

In [ ]:
!nnUNet_plan_and_preprocess -t 006 --verify_dataset_integrity

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2 Task006_Lung 0

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2 Task006_Lung 1

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2 Task006_Lung 2

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2 Task006_Lung 3

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2 Task006_Lung 4

In [ ]:
!nnUNet_train 3d_fullres nnUNetTrainerV2 Task006_Lung 0